# Dash

https://dash.plot.ly/getting-started

Dash is a Python framework for building web apps. It's made by Plotly, so no surprise that it works well for building Dashboards-like apps with analytics and visualization at their core.

With Dash we can make *real* full-service apps for users to do their own exploration and analysis. Here's a good one: https://dash-gallery.plotly.host/dash-yield-curve/

### Same old data

In [18]:
import pandas as pd
DATA_FOLDER = "../../data"
temperatures = pd.read_csv(f"{DATA_FOLDER}/global_temperatures/GlobalLandTemperaturesByCountry.csv", parse_dates=['dt'])
continents = pd.read_csv(f"{DATA_FOLDER}/continents.csv")
countries = pd.read_csv(f"{DATA_FOLDER}/countries.csv")
countries.drop(columns=['code'])
temperatures = temperatures.merge(continents).merge(countries, left_on="Country", right_on="country")

temperatures['year'] = temperatures.dt.dt.year
temperatures['month'] = temperatures.dt.dt.month

yearly_change = temperatures[(temperatures.year==1963) | (temperatures.year==2013)].groupby(["Country","year"], as_index=False).AverageTemperature.mean()
yearly_change['AverageTemperatureChange'] = yearly_change.groupby(["Country"], as_index=False).AverageTemperature.transform("diff")
yearly_change.dropna(inplace=True)
temperature_slice=yearly_change.merge(temperatures[["Country","Continent","Code","lon","lat"]].drop_duplicates())

# The Layout

The layout is the *wireframe* or the *structure* that the app is built on. It is at its core `HTML`, the language of the (backbone of the?) web. HTML elements you may be familiar with are accessible through Python objects. This may seem strange at first, but it makes it so that we can build all of this in one place and one context.

# Who how websites work?

Let's use Chrome Dev Tools to look at how these pieces come together.

- HTML
- CSS
- JS (let's do a jQuery example)

### dash_html_components

The dash_html_components library has a component for every HTML tag. The `html.H1(children='Hello Dash')` component generates a `<h1>Hello Dash</h1>` HTML element in your application.

### dash_core_components

Not all components are pure HTML. The `dash_core_components` describe higher-level components that are interactive and are generated with JavaScript, HTML, and CSS through the React.js library.

In [2]:
from werkzeug.serving import run_simple as _run_simple
def debug(app, port=8000, host="0.0.0.0"): 
    """
    Run a dash app with dash dev tools from within Jupyter Notebooks. 
    
    This behaves the same as running `app.run_server(debug=True)` from ipython directly
    """
    if type(app) == dash.dash.Dash:
        # right, this is a Dash app?
        app.enable_dev_tools()
        app = app.server
    _run_simple(host, port, app)


In [3]:
import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
            ],
            'layout': {
                'title': 'Dash Data Visualization'
            }
        }
    )
])

In [4]:
debug(app)

# EXERCISE

- Replace the figure with one that we like
- Add a Dropdown menu populated with marginal plot types: "violin","rug","box","histogram"

In [5]:
import plotly.express as px

def make_central_figure(marginal_x="violin", marginal_y="histogram"):
    figure = px.scatter(
        temperature_slice,
        x="AverageTemperature",
        y="AverageTemperatureChange",
        color="Continent",
        marginal_y=marginal_y,
        marginal_x=marginal_x,
        hover_data=["Country"]
    )
    return figure
    
def labels_and_values(values):
    return [{'label':val, 'value':val} for val in values]

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=make_central_figure()
    ),
    
    dcc.Dropdown(
        id='marginal-x-dropdown',
        options=labels_and_values(["rug","histogram","violin"])
    )
])

In [6]:
debug(app)

In [ ]:
# Your Solution

# Callbacks

https://dash.plot.ly/getting-started-part-2

This is a term taken primarily from JavaScript, which is the *real* language of the web. Callbacks are triggered when an element or value changes

In `Dash`, we define callbacks using a *function decorator*, `@app.callback()`. This is provided with the `inputs` to watch (e.g. `dash.dependencies.Input(component_id="marginal-x-dropdown", component_property="value")`) and the `outputs` that are updated by the function (`dash.dependencies.Output(component_id="example-graph",component_property="figure")`). To decorate the function we write:

```python
@app.callback(...)
def my_function(input1, input2):
    # determined outputs ...
    return [output1, outpu2]
```

## EXERCISE

- Tie the `value` of the dropdown menu to the `marginal_x` plot drawn

**NOTE:** you need to re-initialize the dash app (`dash.Dash()`) in order to change a callback once it is registered

In [7]:
# Solution

from dash.dependencies import Input, Output

@app.callback(
    output=[Output(component_id="example-graph",component_property="figure")],
    inputs=[Input(component_id="marginal-x-dropdown", component_property="value")]    
)
def redraw_figure_with_marginal(marginal_x):
    figure = make_central_figure(marginal_x=marginal_x)
    return [figure]
debug(app)

In [ ]:
# Your Solution

# Click Events

To make a truly immersive dashboard, we will want to allow users to explore individual data points. This is best done by capturing and handling click events.

## EXERCISE

*We want to use the Continent/Country of the click in order to update a second plot with historical data

- Add a new `html.Div()` element to the page with the `id="output"`. We will use this to print output
- Capture the `clickData` from our main plot and assign it as the `children` value to `output`
- Use this to understand the datastructure of click events

In [76]:
# Solution

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=make_central_figure()
    ),
    
    html.Div(
        id="output",
        children=""
    )
])

@app.callback(
    output=[Output(component_id="output",component_property="children")],
    inputs=[Input(component_id="example-graph", component_property="clickData")]    
)
def printClickdata(clickData):
    return [str(clickData)]

In [ ]:
# Your Solution
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=make_central_figure()
    ),
    
    html.Div(
        # initialize here
    )
])

@app.callback(
    output=[Output(...)],
    inputs=[Input(...)]    
)
def print_clickData(clickData):
    return [str(clickData)]

debug(app)

## EXERCISE

- Build a function to return the average temperature per year for a single country
- Add a second plot beneath the scatter plot with historical data for a single country (this will come from the `temperatures` dataframe)
- Bind this to the `clickData` from the main plot

In [8]:
# Solution
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

def country_plot(countries=[]):
    df = temperatures[temperatures.Country.isin(countries)]
    return px.line(df.groupby(["Country","year"], as_index=False).mean(), x="year", y="AverageTemperature", error_y="AverageTemperatureUncertainty", color="Country")

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=make_central_figure()
    ),
    
    dcc.Graph(
        id='country-graph',
        figure=country_plot()
    )
])

@app.callback(
    output=[Output(component_id="country-graph",component_property="figure")],
    inputs=[Input(component_id="example-graph", component_property="clickData")]
)
def draw_country_history(clickData):
    if clickData:
        points = clickData['points']
        countries = [point['customdata'][0] for point in points]
        figure = country_plot(countries)
        return [figure]

In [ ]:
# Your Solution
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

def country_plot(countries=[]):
    df = None #
    figure = px.line(df)
    return figure

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=make_central_figure()
    ),
    
    dcc.Graph(
        id='country-graph',
        figure=country_plot()
    )
])

@app.callback(
    output=[Output(component_id="country-graph",component_property="figure")],
    inputs=[Input(component_id="example-graph", component_property="clickData")]
)
def draw_country_history(clickData):
    if clickData:
        points = # ...
        countries = # ...
        figure = country_plot(countries)
        return [figure]
debug(app)

## EXERCISE

- Update the `clickData` callback to also support `selectedData`, which can be selected by using the *Box* or *Lasso* tool

In [25]:
# Solution
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

def country_plot(countries=[]):
    df = temperatures[temperatures.Country.isin(countries)]
    return px.line(df.groupby(["Country","year"], as_index=False).mean(), x="year", y="AverageTemperature", error_y="AverageTemperatureUncertainty", color="Country")

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=make_central_figure()
    ),
    
    dcc.Graph(
        id='country-graph',
        figure=country_plot()
    )
])

@app.callback(
    output=[Output(component_id="country-graph",component_property="figure")],
    inputs=[
        Input(component_id="example-graph", component_property="clickData"),
        Input(component_id="example-graph", component_property="selectedData")
    ]
)
def draw_country_history(clickData, selectedData):
    selected = selectedData.get('points',[]) if selectedData else []
    clicked = clickData.get("points",[]) if clickData else []
    points = selected + clicked
    countries = [point['customdata'][0] for point in points]
    figure = country_plot(countries)
    return [figure]
debug(app)

In [ ]:
# Your Solution

# Sliders

Apart from **click events**, visual tools like *dropdowns* and *sliders* are a great way to control and interact with data.  Sliders are a part of the `dash_core_components` package and easier to work with than what we just worked through with `clickData`.


## EXERCISE Bonus

Now that we're working with the full temperature data again:

- Make a dash app with one central figure (something from `px`) that displays a single year of `temperatures` data for all countries
- Control which year is displayed using a slider (`dcc.Slider`)

In [22]:
# Solution
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

def single_year(year=1950):
    df = temperatures[temperatures.year==year].groupby(["Country","year","Continent"]).mean().reset_index()
    df.AverageTemperatureUncertainty.fillna(0,inplace=True)
    return px.scatter(df, x="AverageTemperature", y="AverageTemperatureUncertainty", hover_name="Country", color="Continent", marginal_y="violin",marginal_x="histogram")

app.layout = html.Div(children=[
    html.H1(children='Global Temperatures'),

    html.Div(children='''
        Climate: A single year in the world
    '''),

    dcc.Graph(
        id='year-graph',
        figure=single_year()
    ),
    html.H1(
        id='range-year',
        children='1950'
    ),
    dcc.Slider(
        id='temperature-year',
        min=temperatures.year.min(),
        max=temperatures.year.max(),
        step=1,
        value=1950,
        marks={int(date):str(date) for date in temperatures.year.unique() if date%10==0}
    )
])

@app.callback(
    output=[
        Output(component_id="year-graph",component_property="figure"),
        Output(component_id="range-year",component_property="children")
    ],
    inputs=[
        Input(component_id="temperature-year", component_property="value")
    ]
)
def draw_country_history(year):
    figure = single_year(year)
    return figure, year

#app.run_server(port=8000)
debug(app)

In [ ]:
# Your Solution

# Dashboard Success.